In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
qb = QuantBook()
spy = qb.AddEquity('SPY')
msft = qb.AddEquity('MSFT')
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
# bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
# bbdf.drop('standarddeviation', 1).plot()

In [ ]:
history = history['close'].unstack(level = 0)

In [ ]:
history['spread'] = history['MSFT R735QTJ8XC9X'] - history ['SPY R735QTJ8XC9X']

In [ ]:
history['ideal_action'] = history['spread'].diff().shift(-1).apply(lambda x: 'long' if x < 0 else 'short')

In [ ]:
import altair as alt

alt.Chart(history).mark_point().encode(x = alt.X('SPY R735QTJ8XC9X', scale = alt.Scale(zero = False)), y = alt.Y('MSFT R735QTJ8XC9X', scale = alt.Scale(zero = False)), color = 'ideal_action')

In [ ]:
import numpy as np

history.loc[:, 'approx_return'] = history.loc[:, 'spread'].diff().shift(-1)

In [ ]:
alt.Chart(history).mark_point().encode(x = alt.X('spread', scale = alt.Scale(zero = False)), y = alt.Y('approx_return', scale = alt.Scale(zero = False)), color = 'ideal_action')

In [ ]:
for i in range(1, 60):
    history.loc[:, 'approx_md_return'] = history.loc[:, 'spread'].diff().rolling(i).mean().shift(-i)
    base = alt.Chart(history).mark_point().encode(x = alt.X('spread', scale = alt.Scale(zero = False)), y = alt.Y('approx_md_return', scale = alt.Scale(zero = False)), color = 'ideal_action')
    regression = base.transform_regression('spread', 'approx_md_return').mark_line()
    display(base + regression)

In [ ]:
for i in range(2, 60):
    history.loc[:, 'approx_md_volatility'] = history.loc[:, 'spread'].diff().rolling(i).std().shift(-i)
    base = alt.Chart(history).mark_point().encode(x = alt.X('spread', scale = alt.Scale(zero = False)), y = alt.Y('approx_md_volatility', scale = alt.Scale(zero = False)), color = 'ideal_action')
    regression = base.transform_regression('spread', 'approx_md_volatility').mark_line()
    display(base + regression)

In [ ]:
import statsmodels.api as sm

regressions = pd.DataFrame()
for i in range(2, 45):
    history.loc[:, 'approx_md_return'] = history.loc[:, 'spread'].diff().rolling(i).mean().shift(-i)
    history.loc[:, 'approx_md_volatility'] = history.loc[:, 'spread'].diff().rolling(i).std().shift(-i)
    X = sm.add_constant(history.dropna().loc[:, 'spread'])
    y1 = history.dropna().loc[:, 'approx_md_return']
    y2 = history.dropna().loc[:, 'approx_md_volatility']
    regression1 = sm.OLS(y1, X).fit()
    regression2 = sm.OLS(y2, X).fit()
    regressions.loc[i, 'beta'] = regression1.params.loc['spread']
    regressions.loc[i, 'r-squared'] = regression1.rsquared
    regressions.loc[i, 'volatility beta'] =  regression2.params.loc['spread']
    regressions.loc[i, 'volatility r-squared'] = regression2.rsquared
regressions.head()

In [ ]:
history.loc[:, 'approx_5d_return'] = history.loc[:, 'spread'].diff().rolling(5).mean().shift(-5)
history.loc[:, 'approx_45d_return'] = history.loc[:, 'spread'].diff().rolling(45).mean().shift(-45)

In [ ]:
alt.Chart(history).mark_point().encode(x = 'approx_5d_return', y = 'approx_45d_return', color = 'ideal_action')

In [ ]:
history.loc[:, 'spread'].hist()

In [ ]:
mean = history.loc[:, 'spread'].mean()

In [ ]:
std = history.loc[:, 'spread'].std()

In [ ]:
import math

In [ ]:
history.loc[:, 'PDF'] = (1 / std / math.sqrt(2 * math.pi) * np.exp(- (history.loc[:, 'spread'] - mean) ** 2 / 2 / std ** 2))

In [ ]:
alt.Chart(history).mark_line().encode(x = 'spread', y = 'PDF')

In [ ]:
history.loc[:, 'dist'] = (regression1.params['const'] + history.loc[:, 'spread'] * regression1.params['spread']) * history.loc[:, 'PDF']

In [ ]:
alt.Chart(history).mark_line().encode(x = 'spread', y = alt.Y('dist', scale = alt.Scale(zero = False)))

In [ ]:
regression1.params